In [1]:
#Dùng để get dữ liệu từ Facebook API
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adcreative import AdCreative
import requests

#Dùng để get thời gian
from datetime import datetime, timedelta
import time

#Dùng để work với gg sheet
import pygsheets

#Một số thư viện khác
import pandas as pd
%run LookupValue.ipynb

In [2]:
#Mở gg sheet
client = pygsheets.authorize(service_file = r'C:\Users\Tường\Documents\FACEBOOK-API-BOT--main\maido-agency-e7cc6f08cb95.json')

sh = client.open('MAIDO - DB MAGIC DATA')
wks = sh.worksheet_by_title('AdAccounts')

#Lấy dữ liệu từ CampaignPeriod và lưu thành DataFrame
df = wks.get_as_df()

#Loại bỏ những dòng có access_token là null
drop_rows = df[df['access_token']==''].index
df.drop(drop_rows,axis = 0,inplace = True)

In [3]:
def df_from_insights(insights,access_token):
  data = pd.DataFrame(columns = ['ad_id','date','account_name','campaign_name','adset_name','reach','impressions','cost','post_engagement','message_ads','page_like_ads','post_body','post_url','post_image'])

  #Tạo ra các list rỗng để append giá trị
  date = []
  campaign_name = []
  adset_name = []
  reach = []
  impressions = []
  cost = []
  post_engagement = []
  mess_ads = []
  page_likes_ads = []
  ad_id = []
  account_name = []
  #Lặp qua tất cả các giá trị lấy về được từ get_insights(), nó bao gồm nhiều dict, mỗi dict là một dòng dữ liệu
  for i in insights:
    campaign_name.append(i['campaign_name'])
    adset_name.append(i['adset_name'])
    ad_id.append(i['ad_id'])
    date.append(datetime.strptime(i['date_start'], '%Y-%m-%d')) 
    account_name.append(i['account_name'])
    reach.append(i['reach'])
    impressions.append(i['impressions'])
    cost.append(i['spend'])
    #Lấy ra post_engagement của bài ads, nếu không có => trả về 00
    try:
      value = ""
      for k in i['actions']:
        if k['action_type'] == 'post_engagement':
          value = k['value']
      post_engagement.append(value)
    except KeyError:
      post_engagement.append("")

    #Lấy ra message_ads của bài ads, nếu không có => trả về 00
    try:
      value = ""
      for k in i['actions']:
        if k['action_type'] == 'onsite_conversion.messaging_conversation_started_7d':
          value = k['value']
      mess_ads.append(value)
    except KeyError:
      mess_ads.append("")

    #Lấy ra page_likes_ads của bài ads, nếu không có => trả về 00
    try:
      value = ""
      for k in i['actions']:
        if k['action_type'] == 'like':
          value = k['value']
      page_likes_ads.append(value)
    except KeyError:
      page_likes_ads.append("")

  #Gom tất cả lại thành 1 DataFrame
  df = pd.DataFrame({"campaign_name":campaign_name,
                      "adset_name":adset_name,
                      "account_name": account_name,
                      "ad_id":ad_id,
                      "date":date,
                      "reach":reach,
                      "impressions":impressions,
                      "cost":cost,
                      "post_engagement":post_engagement,
                      "message_ads":mess_ads,
                      "page_like_ads":page_likes_ads}, 
                      columns = ['ad_id','date','account_name','campaign_name','adset_name','reach','impressions','cost','post_engagement','message_ads','page_like_ads'])

  #Tạo list rỗng để append các giá trị
  ad_body = []
  post_url = []
  image_url = []

  #Lấy về các thông tin bằng cách requests graph api của Facebook, tham khảo thêm tại https://developers.facebook.com/docs/graph-api/
  for i in df['ad_id']:
    query = '''https://graph.facebook.com/v15.0/{ad_id}?fields=adcreatives{body,effective_object_story_id,thumbnail_url}
              &access_token={access_token}'''

    run_query = query.replace("{ad_id}",str(i)).replace('{access_token}',str(access_token))
    r = requests.get(run_query)     #Thay id của từng bài post vào để requests
    adcreatives = r.json()          #Thêm .json() để trả về các chuỗi có cả emoji
    #Lấy ra ad_body bằng cách index vào dictionary và thêm vào list 
    try:
      ad_body.append(adcreatives['adcreatives']['data'][0]["body"])
    except KeyError:
      ad_body.append("")

    #Lấy ra post_url bằng cách index vào dictionary và thêm vào list, post_url =  www.facebook.com/{effective_object_story_id}
    try:
      post_url.append("www.facebook.com/"+adcreatives['adcreatives']['data'][0]["effective_object_story_id"])
    except KeyError:
      post_url.append("")

    #Lấy ra image_url bằng cách index vào dictionary và thêm vào list, vì chưa lấy được image nên dùng thumbnail_url để thay thế 
    try:
      image_url.append(adcreatives['adcreatives']['data'][0]["thumbnail_url"])
    except KeyError:
      image_url.append("")
  

  #Nối list thành 3 cột trong DataFrame
  df['post_body'] = ad_body
  df['post_url'] = post_url
  df['post_image'] = image_url
  data = pd.concat([data,df], axis =0)
  return data

In [4]:
def get_data(id, app_id, app_secret,access_token,today_date):

  output = pd.DataFrame(columns = ['date','account_name','campaign_name','adset_name','reach','impressions','cost','post_engagement','message_ads','page_like_ads','post_body','post_url','post_image'])

  #Gọi Facebook API
  FacebookAdsApi.init(app_id, app_secret, access_token)
  print("init")
  
  #Các trường mong muốn trả về
  fields = [
  'account_name',             #Tên tài khoản
  'campaign_name',            #Tên campaign
  'adset_name',               #Tên adset
  'ad_id',                    #Mã bài ad => Dùng để lấy về body, url, image_url
  'actions',                  #Tương tác bài ads, có nhiều trường nhưng chỉ sử dụng post_engagement, onsite_conversion.messaging_conversation_started_7d, likes
  'reach',                    #Reach
  'impressions',              #Impressions
  'spend']                    #Cost
    
  #Các parameters truyền vào
  params = {
      'level': 'ad',                      #lấy dữ liệu theo level: có 4 level là adaccount, campaign, adset, ad. Cái này đang lấy theo level ad
      'limit':10000,                      #giới hạn số lượng kết quả trả về
      'time_range[since]':today_date,     #Ngày bắt đầu
      'time_range[until]':today_date,     #Ngày kết thúc
  }

  #Lấy insights
  insights = AdAccount(id).get_insights(
      params = params,
      fields = fields
  )
  print('Lượng Data:',len(insights))


  #Nếu insights > 0 thì lấy dữ liệu và lưu thành DataFrame
  if len(insights) >0:
    today_data = df_from_insights(insights,access_token)
    print('Đã lấy về được:',len(today_data))
    output = today_data

  #Sửa đổi thứ tự cột trước khi return
  return output[['date','account_name','campaign_name','adset_name','post_body','post_url','post_image','reach','impressions','cost','post_engagement','message_ads','page_like_ads']]

In [6]:
now = datetime.today()
today_date = now.strftime('%Y-%m-%d')
now_non_timestamp = datetime.strptime(today_date,'%Y-%m-%d')
print("Bắt đầu lấy dữ liệu FB Ads -", now)

output = pd.DataFrame(columns = ['date','account_name','campaign_name','adset_name','post_body','post_url','post_image','reach','impressions','cost','post_engagement','message_ads','page_like_ads'])

#Lặp qua các account trong DataFrame để lấy dữ liệu
for i in range(0,len(df)):
  print('Đang lấy data từ:',df.iloc[i]['account_name'])

  #Gọi hàm get_data để lấy dữ liệu
  adaccount_data = get_data(df.iloc[i]['account_id'],df.iloc[i]['app_id'],df.iloc[i]['app_secret'],df.iloc[i]['access_token'],today_date)
  


  #Nối dữ liệu vừa tìm được vào output
  output = pd.concat([output,adaccount_data], axis =0)
 
  print('Xong account:', df.iloc[i]['account_name'], '- Lượng Data:', len(adaccount_data))
  print('---------------------')
  print('')
#Lấy các data khác
output['cost'] = [str(cost).replace(',','') if cost != '' else 0 for cost in output['cost']]
output['cost'] = output['cost'].astype('float')

output['result'] = get_result(output)
output['month'] = get_month(output)
output['service_code'] = get_service_code(output)
output = get_from_settings(output)
print(len(output))
output.drop_duplicates(inplace = True)
output = get_page_data(output)
print(len(output))

output.drop_duplicates(inplace = True)

output['phone_number'] = ["0"+str(i) for i in output['phone_number']]
output.fillna(0.0,inplace=True)

#Lấy ra vị trí cuối cùng của ngày hôm qua để ghi vào
print('Không chỉnh sửa trong thời gian này')
sh1 = client.open('MAIDO - DB MAGIC DATA')


fb_ads_sheet = sh1.worksheet_by_title('Data Raw')
fb_ads = fb_ads_sheet.get_as_df()
fb_ads['date'] = pd.to_datetime(fb_ads['date'],format = '%Y-%m-%d')

yesterday_len = len(fb_ads[fb_ads['date']< now_non_timestamp])
append_index = "A"+str(yesterday_len + 2)
print("Thêm data vào vị trí:", append_index)

columns_order = ['service_code','page_name','month','date','account_name','campaign_name','adset_name','post_body','post_url','post_image','reach','impressions','cost','post_engagement','message_ads','page_like_ads','message_page','page_like_organic','cost_external','phone_number','result']

fb_ads_sheet.set_dataframe(output[columns_order], start=append_index, copy_head=False)

columns_order = ['service_code','page_name','month','date','account_name','campaign_name','adset_name','post_body','post_url','post_image','reach','impressions','cost','post_engagement','message_ads','page_like_ads','cost_external','phone_number','result']
sh2 = client.open('MAIDO - ADS PERFORMANCE - DATA')
ads_performance = sh2.worksheet_by_title('2. Data Raw - FB Ads')
ads_performance.set_dataframe(output[columns_order],start = "A2",copy_head =False)

#Xong rồi đó, nghỉ 30 phút rồi chạy lại
print('------DONE------')
print("Nghỉ 1 tiếng rồi chạy tiếp") 
print('')

Bắt đầu lấy dữ liệu FB Ads - 2023-02-08 16:16:22.192604
Đang lấy data từ: Maido Agency Co., LTD
init
Lượng Data: 102
Đã lấy về được: 102
Xong account: Maido Agency Co., LTD - Lượng Data: 102
---------------------

Đang lấy data từ: Trang Ho
init
Lượng Data: 0
Xong account: Trang Ho - Lượng Data: 0
---------------------

Đang lấy data từ: Công Ty TNHH XNK Á Âu
init
Lượng Data: 24
Đã lấy về được: 24
Xong account: Công Ty TNHH XNK Á Âu - Lượng Data: 24
---------------------

Đang lấy data từ: Maido Agency
init
Lượng Data: 0
Xong account: Maido Agency - Lượng Data: 0
---------------------

Đang lấy data từ: The Ruong
init
Lượng Data: 0
Xong account: The Ruong - Lượng Data: 0
---------------------

Đang lấy data từ: Khoi Nguyen
init
Lượng Data: 0
Xong account: Khoi Nguyen - Lượng Data: 0
---------------------

Đang lấy data từ: Tan Nguyen
init
Lượng Data: 0
Xong account: Tan Nguyen - Lượng Data: 0
---------------------

Đang lấy data từ: Công Ty TNHH Quảng Cáo Maido
init
Lượng Data: 0
Xong 

In [ ]:
# fb_ads_sheet.set_dataframe(output[columns_order], start=append_index, copy_head=False)


In [ ]:
# output[columns_order].to_excel('backup.xlsx')